<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2


    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61


    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-13 16:07:20
-------------------
qualified stocks: 94
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.03 L
Current:  1.27 C
-------------------
Today PnL: 34.91 K (0.28%)
Current PnL: -18.93 L (-13.42%)
CY Booked + Current PnL: -16.06 L (-11.39%)
-------------------
Total profit:  3.73 L
Total loss:  -22.66 L
-------------------
Total Booked + Current PnL: 10.92 L (9.58%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.26%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 80.81 L (63.76%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.42 C (112.36%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 7.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
53         MRF  114777.60  150504.14    14.58       X-MC  90.43  140425.0   
56  PIDILITIND    2504.06    3576.00   -16.01       X-LC  54.26   93018.0   
69     SBILIFE    1496.49    1924.99    -1.57       H-LC  82.98  146160.0   
70        SBIN     760.30     863.00   -13.45       M-LC  59.57  212464.0   
25       DMART    3484.00    5112.00   -20.76       X-LC  13.83   97538.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
53      25647.0  10083.0        0.36         22.35   7.18  31.13    215.0   
56      17896.0  14260.0        0.78         23.82  15.33  42.81     87.0   
69      20455.0  15537.0       -0.66         16.27  10.63  28.63    116.0   
70      10984.0  16232.0        0.01          5.45   7.64  13.51     91.0   
25      13922.0  25145.0        1.01         16.65  25.78  46.73    123.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
53     2.54        1.14    36.79       AR      ATH      TYRES  
56     1.25        0.76    16.66      X40      BTT  CHEMICALS  
69     1.32        1.19    30.21       AR      ATH  INSURANCE  
70     0.68        1.73    16.40     XY25      NTT      BANKS  
25     0.55        0.79    19.39       AR      NTT       FMCG

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
78       TANLA   917.30  1963.11   -40.61       H-SC  95.74  145468.0   
29  GREENPANEL   375.16   537.00   167.44       M-SC  46.81  119428.0   
63   REPCOHOME   515.07   880.00   -56.04       H-SC  81.91  161668.0   
76  STARHEALTH   551.56   761.00   -13.64       M-SC  38.30  141352.0   
77    SURYODAY   189.78   240.00    42.54       H-SC  67.02  100693.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
78     -93947.0  366899.0       12.72        -39.24  252.22  114.01     29.0   
29     -63650.0  142633.0        5.57        -34.77  119.43   43.14     98.0   
63     -43330.0  188570.0        4.37        -21.14  116.64   70.85    130.0   
76     -58864.0  134892.0        4.28        -29.40   95.43   37.97     46.0   
77     -49233.0   88902.0        3.63        -32.84   88.29   26.46    132.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
78    -0.26        1.19    33.42       AR      ATH         IT  
29    -0.45        0.97    12.34     XY24      NTT       MISC  
63    -0.23        1.32    30.51     XY24      NTT    FINANCE  
76    -0.44        1.15    13.49     XY24      NTT  INSURANCE  
77    -0.55        0.82    29.00       XR      NTT      BANKS

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
40        INFY  1432.81  2275.00   -12.42       X-LC  77.66  218035.0   
81         TCS  3928.29  4998.00   -15.84       X-LC  74.47  189810.0   
30  HAPPSTMNDS   919.80  1491.84   -26.03       M-SC  53.19   62302.0   
39  INDUSINDBK  1354.50  1800.00   -40.59       H-MC  56.38   43896.0   
75    SONACOMS   578.05  1006.00   -31.29       M-MC  76.60   88926.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
40      18874.0  98181.0       -3.58          9.48   45.03  58.78      9.0   
81     -22318.0  80081.0       -2.91        -10.52   42.19  27.23      8.0   
30     -34277.0  94344.0       -2.68        -35.49  151.43  62.19     22.0   
39     -33310.0  58707.0       -2.33        -43.14  133.74  32.89     41.0   
75     -12233.0  87121.0       -2.24        -12.09   97.97  74.03     21.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
40     0.19        1.78    13.83      X40      BTT       IT  
81    -0.28        1.55     8.77      X40      BTT       IT  
30    -0.36        0.51     9.24       AR      ATH       IT  
39    -0.57        0.36    20.89       XR      NTT    BANKS  
75    -0.14        0.72    24.98       AR      BTT     AUTO

In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
80  TATAMOTORS   776.02  1065.00   -53.26       M-LC  87.23  187540.0   
2          ABB  5319.37  8953.12   -38.41       H-LC  73.40  140912.0   
24         DLF   683.35  1031.70   -26.39       H-LC  42.55  122013.0   
10         AWL   342.88   485.00   -69.16       H-MC   5.32  167068.0   
92   WHIRLPOOL  1151.42  2270.00   -36.18       M-SC  92.55  160782.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
80     -18105.0   94689.0       -1.82         -8.80  50.49  37.24      1.0   
2        7928.0   82913.0        0.89          5.96  58.84  68.31      2.0   
24       -990.0   63691.0       -0.39         -0.80  52.20  50.98      3.0   
10     -50661.0  140905.0        0.06        -23.27  84.34  41.45      5.0   
92      14552.0  127500.0       -0.41          9.95  79.30  97.15      7.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
80    -0.19        1.53    22.07     XY24      NTT        AUTO  
2      0.10        1.15    14.90       AR      ATH  ELECTRICAL  
24    -0.02        0.99    11.48       AR      ATH      REALTY  
10    -0.36        1.36    11.60     XY24      NTT        FMCG  
92     0.11        1.31    37.35      X40      NTT    DURABLES

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
53         MRF  114777.60  150504.14    14.58       X-MC  90.43  140425.0   
52     MEDANTA    1022.60    1486.00     7.09       X-MC  14.89  188723.0   
56  PIDILITIND    2504.06    3576.00   -16.01       X-LC  54.26   93018.0   
57     POLYCAB    5005.92    7541.35     7.45       H-MC  91.49  168210.0   
4   ADANIPORTS    1103.69    1583.00    -2.94       M-LC  85.11  187580.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
53      25647.0  10083.0        0.36         22.35   7.18  31.13    215.0   
52      33288.0  37141.0        1.18         21.42  19.68  45.32    198.0   
56      17896.0  14260.0        0.78         23.82  15.33  42.81     87.0   
57      28044.0  42944.0        0.25         20.01  25.53  50.65     54.0   
4       36374.0  29281.0        0.52         24.06  15.61  43.43     37.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
53     2.54        1.14    36.79       AR      ATH       TYRES  
52     0.90        1.54    28.41     XY24      NTT  HEALTHCARE  
56     1.25        0.76    16.66      X40      BTT   CHEMICALS  
57     0.65        1.37    28.38     X40N      ATH      CABLES  
4      1.24        1.53    30.02     XY24      NTT        MISC

In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
57     POLYCAB  5005.92  7541.35     7.45       H-MC  91.49  168210.0   
6     ANGELONE  2328.67  3033.00    -0.62       H-SC  69.68  161694.0   
4   ADANIPORTS  1103.69  1583.00    -2.94       M-LC  85.11  187580.0   
93       WIPRO   243.46   420.00    -8.80       M-LC  50.00  155973.0   
70        SBIN   760.30   863.00   -13.45       M-LC  59.57  212464.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
57      28044.0   42944.0        0.25         20.01  25.53  50.65     54.0   
6       10330.0   35443.0        0.02          6.82  21.92  30.25    131.0   
4       36374.0   29281.0        0.52         24.06  15.61  43.43     37.0   
93       5028.0  104424.0       -2.22          3.33  66.95  72.51     58.0   
70      10984.0   16232.0        0.01          5.45   7.64  13.51     91.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
57     0.65        1.37    28.38     X40N      ATH   CABLES  
6      0.29        1.32    27.98     X40N      NTT  FINANCE  
4      1.24        1.53    30.02     XY24      NTT     MISC  
93     0.05        1.27    17.44       XR      NTT       IT  
70     0.68        1.73    16.40     XY25      NTT    BANKS

In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
93      WIPRO   243.46   420.00    -8.80       M-LC  50.00  155973.0   
37  INDIAMART  2311.97  4911.36   -52.21       H-SC  78.72  103418.0   
18       CAMS  3643.00  5250.99    -9.78       H-SC   6.38  101534.0   
12  BANKINDIA   113.49   190.00   -34.31       M-MC  34.04  128093.0   
91     VOLTAS  1272.00  1929.20   -30.74       H-MC  15.96  138347.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
93       5028.0  104424.0       -2.22          3.33   66.95   72.51     58.0   
37       1691.0  112684.0        2.49          1.66  108.96  112.43      NaN   
18       -470.0   45497.0       -1.24         -0.46   44.81   44.14     47.0   
12      -1059.0   88128.0        2.24         -0.82   68.80   67.42     71.0   
91      -1573.0   73863.0       -0.63         -1.12   53.39   51.67     19.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
93     0.05        1.27    17.44       XR      NTT       IT  
37     0.02        0.84    22.06       AR      ATH     MISC  
18    -0.01        0.83    17.72       XR      ATH     MISC  
12    -0.01        1.04    24.58       XR      NTT    BANKS  
91    -0.02        1.13     4.49       AR      ATH       AC

In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
85  UJJIVANSFB      52.77      60.00    36.52       M-SC  79.79  118719.0   
53         MRF  114777.60  150504.14    14.58       X-MC  90.43  140425.0   
69     SBILIFE    1496.49    1924.99    -1.57       H-LC  82.98  146160.0   
86   UNIONBANK     123.87     163.00   -15.64       M-LC  68.09  148685.0   
4   ADANIPORTS    1103.69    1583.00    -2.94       M-LC  85.11  187580.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
85     -23760.0  43285.0        2.16        -16.68  36.46  13.70    219.0   
53      25647.0  10083.0        0.36         22.35   7.18  31.13    215.0   
69      20455.0  15537.0       -0.66         16.27  10.63  28.63    116.0   
86       7845.0  36651.0        2.08          5.57  24.65  31.59     83.0   
4       36374.0  29281.0        0.52         24.06  15.61  43.43     37.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
85    -0.55        0.97    40.12     X40N      NTT      BANKS  
53     2.54        1.14    36.79       AR      ATH      TYRES  
69     1.32        1.19    30.21       AR      ATH  INSURANCE  
86     0.21        1.21    29.16     XY24      NTT      BANKS  
4      1.24        1.53    30.02     XY24      NTT       MISC

In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
90     VIPIND   488.80   718.00  -810.92       H-SC  100.00   67694.0   
58      QUESS   711.66   986.00      NaN        NaN   98.94   79585.0   
79  TATAELXSI  7332.28  9000.00   -20.18       X-MC   97.87   83804.0   
1    AARTIIND   487.04   919.00    35.69       M-SC   96.81   77939.0   
78      TANLA   917.30  1963.11   -40.61       H-SC   95.74  145468.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
90     -27622.0   72318.0        0.93        -28.98  106.83   46.89     51.0   
58     -76269.0  136353.0        2.37        -48.94  171.33   38.55      NaN   
79     -18848.0   42195.0       -1.87        -18.36   50.35   22.74     33.0   
1       -3397.0   75531.0       -0.41         -4.18   96.91   88.69    118.0   
78     -93947.0  366899.0       12.72        -39.24  252.22  114.01     29.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
90    -0.38        0.55    33.52     X40N      NTT       MISC  
58    -0.56        0.65    21.86     XY24      NTT       MISC  
79    -0.45        0.68    25.97     X40N      NTT         IT  
1     -0.04        0.64    31.43       XR      NTT  CHEMICALS  
78    -0.26        1.19    33.42       AR      ATH         IT

In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.30
1     25       41.06
2     50       69.55

In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.79
X40     14.50
X40N    14.03
AR      13.83
XR      10.31
XY25    10.27
X200     1.20
SR       1.10
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.33
X-LC    18.29
M-SC    16.59
M-LC     8.43
H-MC     7.86
X-MC     6.81
H-LC     5.67
M-MC     5.46
X-SC     2.82
L-SC     2.26
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                11.82         -2.32   34.86
BANKS               11.10        -10.61   64.87
IT                   9.28        -21.53   89.33
FINANCE              7.46        -20.98   75.57
MISC                 7.22        -27.88   82.91
INSURANCE            6.09         -3.33   34.48
PAINTS               5.60        -19.70   55.13
ELECTRICAL           4.13         -9.88   65.87
HEALTHCARE           3.66         -3.68   37.16
REFINERIES           3.65          8.98    7.79
CHEMICALS            3.29        -17.27   98.07
AUTO                 3.19        -15.93   67.82
FOOTWEAR             2.40        -39.51   85.61
DURABLES             1.99         -1.36   80.07
TRAVEL               1.95        -32.87   90.37
AC                   1.93        -13.39   73.88
CERAMICS             1.68        -48.19  110.76
TEXTILES             1.67        -11.83   70.57
STEEL                1.63        -12.62   96.55
CABLES               1.37         16.67   25.53
JEWELLERY            1.27        -86.01  144.83
TYRES                1.14         18.26    7.18
CEMENT               1.10        -39.15  110.75
PHARMA               1.09          2.03   46.12
ENGINEERING          1.07        -46.47  106.65
REALTY               0.99         -0.81   52.20
ENTERTAINMENT        0.91        -46.15  119.65
POWER                0.89        -29.71   81.44
HOTELS               0.46       -100.54  171.28

In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3083829.0     27
XR        1279413.0     15
AR        1257377.0     15
X40N      1212175.0     17
X40        815894.0     12
SR         209859.0      2
XY25       151655.0      5
X200        70280.0      1

In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        2451000.0     22
M-SC        1970761.0     20
X-LC         740051.0     12
M-MC         596734.0      5
H-MC         431985.0      7
X-SC         363821.0      4
M-LC         345148.0      6
X-MC         319887.0      6
L-SC         319822.0      4
H-LC         229922.0      5

In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      937582.0      6
M-SC       XY24      776464.0      6
H-SC       X40N      562910.0      7
           AR        541467.0      3
M-SC       XR        434502.0      5
M-MC       XY24      421485.0      3
X-LC       X40       379513.0      5
M-SC       X40N      377314.0      5
H-SC       XR        306869.0      4
M-LC       XY24      224492.0      4
M-SC       X40       201635.0      2
H-LC       AR        194843.0      4
X-LC       AR        189914.0      3
H-MC       XY24      184360.0      2
L-SC       XR        165022.0      2
X-MC       XY24      139037.0      2
X-SC       SR        123357.0      1
           XR        121761.0      1
           X40N      118703.0      2
X-MC       X40N      110304.0      2
M-LC       XR        104424.0      1
H-SC       X40       102172.0      2
X-LC       XY25      100344.0      3
M-SC       AR         94344.0      1
L-SC       XY24       89058.0      1
M-MC       XR         88128.0      1
           AR         87121.0      1
M-SC       SR         86502.0      1
H-MC       AR         73863.0      1
           X40        72111.0      2
X-LC       X200       70280.0      1
L-SC       AR         65742.0      1
X-MC       X40        60463.0      1
H-MC       XR         58707.0      1
           X40N       42944.0      1
H-LC       XY25       35079.0      1
M-LC       XY25       16232.0      1
X-MC       AR         10083.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
